# Shared Memory: Threads and OpenMP

- Shared memory is based on the thread model where threads (lightweight processes) share the same global memory.
- There is no need for communication
- Creating and destroying threads should be easy/cheap -> Fork/Join model
- Limited by local memory
  <div style="text-align: center;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/f/f1/Fork_join.svg" alt="Image Description" width="800">
</div>


<div style="text-align: center;">
    <img src="https://passlab.github.io/OpenMPProgrammingBook/_images/2.png" alt="Image Description" width="600">
</div>

- [Introduction to OpenMP - Tim Mattson - Intel](https://www.youtube.com/watch?v=nE-xN4Bf8XI&list=PLLX-Q6B8xqZ8n8bwjGdzBJ25X2utwnoEG)

- https://passlab.github.io/OpenMPProgrammingBook/MultiCoreMultiCPU/1_MIMDArchitecture.html

## C++ example
[OpenMP](https://www.openmp.org/) is the standard for using threads in HPC, for C/C++/Fortran. But you can also use [Posix threads](https://en.wikipedia.org/wiki/Pthreads?useskin=vector), [`std::threads`](https://en.cppreference.com/w/cpp/thread) from c++, an so on.

In [19]:
%%writefile openmp.cpp
#include <iostream>
#include <omp.h>

int main(int argc, char *argv[]) {
  std::cout << "BEFORE\n";
#pragma omp parallel
  {
    int thid = omp_get_thread_num();
    std::cout << "Hello world, from " << thid << "\n";
  }
  std::cout << "AFTER\n";
    
  return 0;
}

Overwriting openmp.cpp


In [20]:
# Compilation
!g++-13 -fopenmp openmp.cpp -o openmp.x 
# execution
!OMP_NUM_THREADS=5 ./openmp.x

BEFORE
Hello world, from Hello world, from Hello world, from Hello world, from Hello world, from 1
4
0
3
2
AFTER


## Python example
For Python, you can use the [threading](https://docs.python.org/3/library/threading.html#module-threading) (The  [Multiprocessing](https://docs.python.org/3/library/multiprocessing.html) module actually generates processes).

In [5]:
import threading
import time

# Define a function that will be executed in a separate thread
def task(name):
    print(f"Task {name} started")
    time.sleep(3)  # Simulate some time-consuming task
    print(f"Task {name} completed")

# Create multiple threads and start them
threads = []
for i in range(5):
    t = threading.Thread(target=task, args=(f"Thread-{i+1}",))
    threads.append(t)
    t.start()

# Allow some tasks to execute before starting others
time.sleep(1)

print("Main thread continues executing")

# Wait for all threads to finish
for t in threads:
    t.join()

print("All threads completed")


Task Thread-1 started
Task Thread-2 started
Task Thread-3 started
Task Thread-4 started
Task Thread-5 started
Main thread continues executing
Task Thread-2 completedTask Thread-3 completed
Task Thread-4 completed
Task Thread-5 completed
Task Thread-1 completed

All threads completed


# OpenMP in C++

<div style="text-align: center;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/9/9b/OpenMP_language_extensions.svg" alt="Image Description" width="800">
</div>

The [`openmp parallel`](https://www.openmp.org/spec-html/5.0/openmpse14.html) directive
```c++
#pragma omp parallel [clause ...]  newline 
                   if (scalar_expression) 
                   private (list) 
                   shared (list) 
                   default (shared | none) 
                   firstprivate (list) 
                   reduction (operator: list) 
                   copyin (list) 
                   num_threads (integer-expression)
 structured_block  

```

In [ ]:
int main(void)
{
    double x = 0;
    {
        double x = 20;
    }
    x += 10.0;
}

In [1]:
!nproc
!free -m

10
zsh:1: command not found: free


# Example: Computing average of a large vector
Let's assume that we have a large vector and we want to compute the average of its elements. First, think on how to parallelize the problem by using domain/problem decomposition
<div style="text-align: center;">
<table>
  <tr>
    <td>
        <div style="text-align: center;">
    <img src="https://nyu-cds.github.io/python-mpi/fig/04-domain-decomp.png" alt="Image Description" width="400">
</div>
    </td>
    <td>
    <div style="text-align: center;">
    <img src="https://nyu-cds.github.io/python-mpi/fig/04-partitions.png" alt="Image Description" width="400">
</div>  
    </td>
  </tr>
      <tr>
    <td>
        <div style="text-align: center;">
    <img src="https://stomp-userguide.pnnl.gov/estomp_guide/images/domain_decomposition.png" alt="Image Description" width="400">
</div>
    </td>
    <td>
    <div style="text-align: center;">
    <img src="https://fun3d.larc.nasa.gov/dpw_near_grid.png" alt="Image Description" width="400">
</div>  
    </td>
  </tr>

</table>
</div>


## Domain decomposition: by hand
Imagine that you have a vector of size $N$. And you have $nth$ threads. How will you split the work among them? What would be the limits for each thread? What variables should be private?

In [22]:
%%writefile avg_openmp.cpp
//# Solution: Manually splitting the domain
#include <omp.h>
#include <vector>
#include <iostream>

void init(std::vector<double> & array);

int main(int argc, char **argv) {
    // resources
    const int N = std::stoi(argv[1]);
    std::vector<double> data(N, 0.0);
    init(data);
    double suma_total = 0.0;

    double start = omp_get_wtime();
    // calcular las sumas parciales
#pragma omp parallel 
    {
        int thid = omp_get_thread_num();
        int nth = omp_get_num_threads();
        int Nlocal = N/nth;

        double suma = 0.0;
        for(int ii = Nlocal*thid; ii < (thid+1)*Nlocal; ++ii) {
            suma += data[ii];
        }
        //std::cout << thid << "\t" << suma << "\n";
	#pragma omp atomic update
	suma_total += suma;
    }
    double end = omp_get_wtime();
    std::cout << end-start << "\n";
    
    // imprimir la informacion
    std::cout << "avg: " << suma_total/N << "\n";
    
    return 0;
}

void init(std::vector<double> & array)
{
    for (auto & x : array) {
        x = 1.7;
    }
}

Fortunately, `OpenMP` already does all the heavy-lifting for us, using the `parallel for` directive

In [23]:
%writefile avg_parallelfor.cpp
# Solution using parallel for
#include <omp.h>
#include <vector>
#include <iostream>

void init(std::vector<double> & array);

int main(int argc, char **argv) {
    // resources
    const int N = std::stoi(argv[1]);
    std::vector<double> data(N, 0.0);
    init(data);

    double suma_total = 0.0;

    double start = omp_get_wtime();
    // calcular las sumas parciales
#pragma omp parallel for reduction(+:suma_total)
    for(int ii = 0; ii < N; ++ii) {
        suma_total += data[ii];
    }
    double end = omp_get_wtime();
    std::cout << end-start << "\n";
    
    // imprimir la informacion
    std::cout << "avg: " << suma_total/N << "\n";
    
    return 0;
}

void init(std::vector<double> & array)
{
    for (auto & x : array) {
        x = 1.7;
    }
}

## Parallel metrics
Compute the parallel metrics (speedup and parallel efficiency). Do you notice something special when reaching the number of cores? Use a small and a large system size. Use `slurm` to launch the jobs. Use the script builders or https://researchcomputing.princeton.edu/support/knowledge-base/scaling-analysis


## Python implementation


In [2]:
# From chat gpt: Problematic, result depends on the number of threads
import threading
import numpy as np

# Define the number of elements in the vector and the number of threads
vector_size = 1000000000
num_threads = 10

# Generate a large vector of random numbers
np.random.seed(10)
vector = np.random.rand(vector_size)

# Define a function that each thread will execute to compute the partial sum
def compute_partial_sum(start, end, partial_sums):
    partial_sum = np.sum(vector[start:end])
    partial_sums.append(partial_sum)

# Create a list to store the partial sums
partial_sums = []

# Create and start the threads
threads = []
chunk_size = vector_size // num_threads
for i in range(num_threads):
    start = i * chunk_size
    end = start + chunk_size
    t = threading.Thread(target=compute_partial_sum, args=(start, end, partial_sums))
    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

# Compute the final mean
mean = np.mean(partial_sums)

print("Mean:", mean)


Mean: 50001490.589945585


In [8]:
# "Fix" with a lock (problem persists)
import threading
import numpy as np

# Define the number of elements in the vector and the number of threads
vector_size = 10000000
num_threads = 5

# Generate a large vector of random numbers
np.random.seed(10)
vector = np.random.rand(vector_size)

# Define a function that each thread will execute to compute the partial sum
def compute_partial_sum(start, end, partial_sums, lock):
    partial_sum = np.sum(vector[start:end])

    # Acquire the lock before modifying the shared list
    lock.acquire()
    partial_sums.append(partial_sum)
    lock.release()

# Create a list to store the partial sums
partial_sums = []

# Create a lock for synchronizing access to the partial_sums list
lock = threading.Lock()

# Create and start the threads
threads = []
chunk_size = vector_size // num_threads
for i in range(num_threads):
    start = i * chunk_size
    end = start + chunk_size

    t = threading.Thread(target=compute_partial_sum, args=(start, end, partial_sums, lock))
    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

# Compute the final mean
mean = np.mean(partial_sums)

print("Mean:", mean)


Mean: 1000247.2920411301


In [10]:
# chatGPT never got the fix: to compute the average with the total number of elements
import threading
import numpy as np

# Define the number of elements in the vector and the number of threads
vector_size = 10000000
num_threads = 2

# Generate a large vector of random numbers
np.random.seed(10)
vector = np.random.rand(vector_size)

# Define a function that each thread will execute to compute the partial sum
def compute_partial_sum(start, end, partial_sums):
    partial_sum = np.sum(vector[start:end])
    partial_sums.append(partial_sum)

# Create a list to store the partial sums
partial_sums = []

# Create and start the threads
threads = []
chunk_size = vector_size // num_threads
for i in range(num_threads):
    start = i * chunk_size
    end = start + chunk_size
    t = threading.Thread(target=compute_partial_sum, args=(start, end, partial_sums))
    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

# Compute the final mean
mean = np.mean(partial_sums)
print("Mean:", mean)

# This is the real problem, chatGPT does not understand it
mean = np.sum(partial_sums)/vector_size
print("Mean:", mean)



Mean: 2500618.230102826
Mean: 0.5001236460205651


# Workshop
## Mean and standard deviation
Write a program to compute the mean and the standard deviation for a large array using OpenMP, with only one reduction. Also compute the parallel metrics. 

## Integral
Write a program to compute the integral of the function , for $ x \in [0, 10]$. Fill the following table:
   |Threads|Runtime[s]|Speedup|Efficiency|
   |---|---|---|---|
   |||||
  
Use a constant and large $N$. What if you distribute $N$ among threads? what if you keep $N$ constant per thread?
   

## Matrix-matrix multiplication
Parallelize a Matrix-Matrix multiplication. Compare the performance when you use one, two, three, for threads.